In [13]:
!python -V

Python 3.9.12


In [14]:
import pandas as pd

In [15]:
import pickle

In [16]:
import seaborn as sns
import matplotlib.pyplot as plt

In [17]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [18]:
import mlflow

#mlflow.set_tracking_uri("sqlite:///mlflow.db")

# This should be DNS of EC2 instance
mlflow.set_tracking_uri("http://ec2-3-25-88-45.ap-southeast-2.compute.amazonaws.com:5000")
# Create experiment with s3 bucket to store in s3
mlflow.create_experiment("nyc-taxi-experiment5", artifact_location="s3://mlflow-artifacts-remote31/mlflow-artifacts")
mlflow.set_experiment("nyc-taxi-experiment5")

#export MLFLOW_TRACKING_URI=http://ec2-3-27-183-70.ap-southeast-2.compute.amazonaws.com:5000

<Experiment: artifact_location='s3://mlflow-artifacts-remote31/mlflow-artifacts', creation_time=1764090309470, experiment_id='6', last_update_time=1764090309470, lifecycle_stage='active', name='nyc-taxi-experiment5', tags={}>

In [19]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [20]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')

In [21]:
len(df_train), len(df_val)

(73908, 61921)

In [22]:
df_train = df_train.iloc[:10000]
df_val = df_val.iloc[:10000]

In [23]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [24]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [25]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [26]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

8.297527100662364

In [27]:
import os
#os.mkdir('models')

with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [28]:
with mlflow.start_run():

    mlflow.set_tag("developer", "am")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

/home/ubuntu/anaconda3/lib/python3.9/site-packages/boto3/compat.py:84: PythonDeprecationWarning: Boto3 will no longer support Python 3.9 starting April 29, 2026. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.10 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


🏃 View run fun-bear-709 at: http://ec2-3-25-88-45.ap-southeast-2.compute.amazonaws.com:5000/#/experiments/6/runs/f4832d438b4e4023a6bbb4fe564a84fa
🧪 View experiment at: http://ec2-3-25-88-45.ap-southeast-2.compute.amazonaws.com:5000/#/experiments/6


In [29]:
#!pip install lightgbm

In [30]:
import lightgbm as lgb

In [31]:
#from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
#from hyperopt.pyll import scope

In [32]:
#train = xgb.DMatrix(X_train, label=y_train)
#valid = xgb.DMatrix(X_val, label=y_val)

In [33]:
"""
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}
"""

'\ndef objective(params):\n    with mlflow.start_run():\n        mlflow.set_tag("model", "xgboost")\n        mlflow.log_params(params)\n        booster = xgb.train(\n            params=params,\n            dtrain=train,\n            num_boost_round=1000,\n            evals=[(valid, \'validation\')],\n            early_stopping_rounds=50\n        )\n        y_pred = booster.predict(valid)\n        rmse = mean_squared_error(y_val, y_pred, squared=False)\n        mlflow.log_metric("rmse", rmse)\n\n    return {\'loss\': rmse, \'status\': STATUS_OK}\n'

In [34]:
'''
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)
'''

"\nsearch_space = {\n    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),\n    'learning_rate': hp.loguniform('learning_rate', -3, 0),\n    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),\n    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),\n    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),\n    'objective': 'reg:linear',\n    'seed': 42\n}\n\nbest_result = fmin(\n    fn=objective,\n    space=search_space,\n    algo=tpe.suggest,\n    max_evals=50,\n    trials=Trials()\n)\n"

In [35]:
#mlflow.xgboost.autolog(disable=True)

In [36]:
'''
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")
'''

'\nwith mlflow.start_run():\n    \n    train = xgb.DMatrix(X_train, label=y_train)\n    valid = xgb.DMatrix(X_val, label=y_val)\n\n    best_params = {\n        \'learning_rate\': 0.09585355369315604,\n        \'max_depth\': 30,\n        \'min_child_weight\': 1.060597050922164,\n        \'objective\': \'reg:linear\',\n        \'reg_alpha\': 0.018060244040060163,\n        \'reg_lambda\': 0.011658731377413597,\n        \'seed\': 42\n    }\n\n    mlflow.log_params(best_params)\n\n    booster = xgb.train(\n        params=best_params,\n        dtrain=train,\n        num_boost_round=1000,\n        evals=[(valid, \'validation\')],\n        early_stopping_rounds=50\n    )\n\n    y_pred = booster.predict(valid)\n    rmse = mean_squared_error(y_val, y_pred, squared=False)\n    mlflow.log_metric("rmse", rmse)\n\n    with open("models/preprocessor.b", "wb") as f_out:\n        pickle.dump(dv, f_out)\n    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")\n\n    mlflow.xgboost

In [37]:
# LightGBM

with mlflow.start_run():
    
    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'regression',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42,
        'metric': 'rmse'
    }

    mlflow.log_params(best_params)

    booster = lgb.train(
        params=best_params,
        train_set=train_data,
        num_boost_round=1000,
        valid_sets=[valid_data],
        valid_names=['validation'],
        callbacks=[lgb.early_stopping(stopping_rounds=50)]
    )

    y_pred = booster.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.lightgbm.log_model(booster, artifact_path="models_mlflow")


[LightGBM] [Warning] Provided parameters constrain tree depth (max_depth=30) without explicitly setting 'num_leaves'. This can lead to underfitting. To resolve this warning, pass 'num_leaves' (<=1073741824) in params. Alternatively, pass (max_depth=-1) and just use 'num_leaves' to constrain model complexity.
[LightGBM] [Warning] Provided parameters constrain tree depth (max_depth=30) without explicitly setting 'num_leaves'. This can lead to underfitting. To resolve this warning, pass 'num_leaves' (<=1073741824) in params. Alternatively, pass (max_depth=-1) and just use 'num_leaves' to constrain model complexity.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002886 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 443
[LightGBM] [Info] Number of data points in the train set: 10000, number of used features: 95
[LightGBM] [Warning] Provided parameters constrain tree depth (max_depth=30) without explici

2025/11/25 17:06:22 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Early stopping, best iteration is:
[80]	validation's rmse: 6.41333


2025/11/25 17:06:29 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run righteous-bear-363 at: http://ec2-3-25-88-45.ap-southeast-2.compute.amazonaws.com:5000/#/experiments/6/runs/c83577e800dc4c528d27d97b5a20cd65
🧪 View experiment at: http://ec2-3-25-88-45.ap-southeast-2.compute.amazonaws.com:5000/#/experiments/6


In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog() # default artifact path is model

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifacts("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        

2025/11/25 17:06:52 WARNING mlflow.utils.autologging_utils: MLflow sklearn autologging is known to be compatible with 1.3.0 <= scikit-learn <= 1.7.0, but the installed version is 1.0.2. If you encounter errors during autologging, try upgrading / downgrading scikit-learn to a compatible version, or try upgrading MLflow.


🏃 View run efficient-zebra-142 at: http://ec2-3-25-88-45.ap-southeast-2.compute.amazonaws.com:5000/#/experiments/6/runs/e540a77b84084cc6a9ea7093b8ad0474
🧪 View experiment at: http://ec2-3-25-88-45.ap-southeast-2.compute.amazonaws.com:5000/#/experiments/6
🏃 View run legendary-shrike-252 at: http://ec2-3-25-88-45.ap-southeast-2.compute.amazonaws.com:5000/#/experiments/6/runs/764260bfa5ec4f84a9b9e462b99a4995
🧪 View experiment at: http://ec2-3-25-88-45.ap-southeast-2.compute.amazonaws.com:5000/#/experiments/6
🏃 View run skillful-owl-924 at: http://ec2-3-25-88-45.ap-southeast-2.compute.amazonaws.com:5000/#/experiments/6/runs/4e7b2218e7164736a0651ad8fa3802ea
🧪 View experiment at: http://ec2-3-25-88-45.ap-southeast-2.compute.amazonaws.com:5000/#/experiments/6


/home/ubuntu/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


🏃 View run clumsy-stork-204 at: http://ec2-3-25-88-45.ap-southeast-2.compute.amazonaws.com:5000/#/experiments/6/runs/28d0aa7a24c44012a3225f006e2fb9ef
🧪 View experiment at: http://ec2-3-25-88-45.ap-southeast-2.compute.amazonaws.com:5000/#/experiments/6


In [39]:
# Check folder 
with mlflow.start_run() as run:
    print("Artifact URI:", run.info.artifact_uri)

Artifact URI: s3://mlflow-artifacts-remote31/mlflow-artifacts/0a0ff62e9b504a17962ded7be10c080f/artifacts
🏃 View run bold-lynx-803 at: http://ec2-3-25-88-45.ap-southeast-2.compute.amazonaws.com:5000/#/experiments/6/runs/0a0ff62e9b504a17962ded7be10c080f
🧪 View experiment at: http://ec2-3-25-88-45.ap-southeast-2.compute.amazonaws.com:5000/#/experiments/6
